In [1]:
from fastai.text.all import *
import pandas as pd


In [6]:
path = untar_data(URLs.IMDB_SAMPLE)
pd.read_csv(path/'texts.csv')

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False
...,...,...,...
995,negative,"There are many different versions of this one floating around, so make sure you can locate one of the unrated copies, otherwise some gore and one scene of nudity might be missing. Some versions also omit most of the opening sequence and other bits here and there. The cut I saw has the on-screen title WITCHCRAFT: EVIL ENCOUNTERS and was released by Shriek Show, who maintain the original US release title WITCHERY for the DVD release. It's a nice-looking print and seems to have all of the footage, but has some cropping/aspect ratio issues. In Italy, it was released as LA CASA 4 (WITCHCRAFT). ...",True
996,positive,"Once upon a time Hollywood produced live-action, G-rated movies without foul language, immorality, and gore-splattered violence. These movies neither insulted your intelligence no manipulated your emotions. The heroes differed little from the crowd. They shared the same feelings and bore the same burdens. Since the 1970s, the film industry has pretty much written off G-rated movies for adults. Basically, modern mature audi

In [7]:
imdb = pd.read_csv(path/'texts.csv')
imdb.is_valid.sum()

200

In [22]:
df = pd.read_csv("feedback-prize-2021/train.csv").rename(columns={"discourse_text":"text", "discourse_type":"label"})
l = df.shape[0]
df["is_valid"] = [False for i in range(int(l * 0.7))] + [True for i in range(l-int(l * 0.7))]
df

,id,discourse_id,discourse_start,discourse_end,text,label,discourse_type_num,predictionstring,is_valid
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving.,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44,False
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stuff happens when it comes to a phone.,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,False
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban phones from class rooms just because of it.,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,False
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138,False
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems.,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162,False
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,"if I'm not sure what college I want to attend, and I ask one friend where I should go, they might say ""George Mason"" as a suggestion. But if George Mason is not in fact the best college for me, even though the person I asked was trying to help, that one opinion could sway me to go there anyway and not have the best experience. But yet if I go to ten people to advice, including the person who suggested George Mason, they might be the only one who would say that and maybe five of the other people I asked might say Virginia Tech, and I would decide to go to Virginia Tech instead of to George...",Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528 529 530 531 532 533 534 ...,True
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,"seeking multiple opinions before making a hard decision can be beneficial because of the reasons stated above. When I was in second grade, I passed the test to be admitted into the AAP (Advanced Academics Program) in my school. My best friend was also accepted, and wanted me to go to AAP with her, but I didnt really want to go into AAP. I told my parents this, and they agreed it would be best for me to stay in the normal class for on more year. Since I got my parents' opinions, I also felt more secure staying out of the advanced program even though my friend wanted me to do the opposite a...",Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609 610 611 612 613 614 615 616 617 618 619 620 6

In [24]:
dls = TextDataLoaders.from_df(df, text_col='text', label_col='label', valid_col='is_valid')
dls.show_batch(max_n=2)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck
Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


,text,category
0,"xxbos xxmaj in the year 1976 , when xxmaj viking 1 space craft was taking photographs of possiable landing spots for xxmaj viking 2 spacecraft , they found a formation in a rock that looked like a human face . xxmaj the shadows made the rock look like it had a nose , mouth , and two eyes , like a human . xxmaj this was found in a region of the xxmaj red xxmaj planet called xxmaj cydonia . xxmaj the "" head "" was nearly two miles from end to end , and it seemed to be starring back at the cameras . \n\n xxmaj the controllers back at the xxmaj jet xxmaj propulsion xxmaj lab were probably quite surprised when this face popped up on their screens , looking them right in the eyes . xxmaj this sensation was short lived . xxmaj scientist figured it was",Evidence
1,"xxbos xxmaj for me a car is very important , because if you have a car you can move to other places that you want . xxmaj but not only that , if you have an emergency you can go rapidly , or in time . xxmaj but this is not a facility in all countries and exist much problems with that , for example in xxmaj germany the street parking , driveways and home garages , this are generally forbidden , new district on the outskirts of xxmaj freiburg , near the xxmaj french and xxmaj swiss . but in xxmaj vauban 's streets are completly car free , this is so good , but with a limit , like ▁ except the main thoroughfare , where the tram to downtown xxmaj freiburg runs . \n\n xxmaj in xxunk a result , 70 percent of xxmaj vauban 's families",Evidence


In [ ]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.2, metrics=[accuracy, Perplexity()]).to_fp16()

In [30]:
learn.fit_one_cycle(1, 2e-2)

C:\Users\Prannaya\.conda\envs\analytics\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:76] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4149510144 bytes.

In [ ]:
learn.save(r"langepoch1")

In [ ]:
learn_txtcls = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)